# RWKV v5 Baseline C - memory finetune

This continues off from `./BaseV5-C-basemodel.ipynb` to perform the full memory finetune & testing process

This is done generally in 3 Tune stages
- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.
- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.
- Tune 3: Mid ctx size (1024), stage 2, scaled up to 1024 context sizes.
- Tune 4: Mid ctx size (4096), stage 3, scaled up to 4096 context sizes.

In all cases, the input tokens is always masked. And we intentionally use the limited word set for memory training, which matches the same wordset used in the original memory evaluation of raven pretrained models. This is intentional to serve as both consistent comparision between experiments, and resonable training time.

One of the issue faced previously with an excessive large word set, is that the model would be required to see "new words" atleast a few time before being able to train the memory process. This drastically slowed down the process as the large word list meant the model was constantly spending time learning new words (instead of memory training).

If we want to increase the number / type of words the model can handle for memory training, that can be done later as a stage 4 memory tune if needed. But that exceeds the current requirements for the memory experiment process.

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [1]:
# # Init required dirs
# !mkdir -p ../../../../model/
# !mkdir -p ../../../../datapath/
# !mkdir -p ../../../../checkpoint/

# # Download the Stage2.pth file
# !rm -rf ../../../../model/BaseV5-C-Stage2.pth
# !cd ../../../../model/ && wget -nc https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/BaseV5-C-Stage2.pth
# !ls -alh ../../../model/BaseV5-C-Stage2.pth

## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="BaseV5-R2-C"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C
INFERENCE_DIR: /root/rwkv-x-playground/RWKV-v5
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5
PROJECT_DIR: /root/rwkv-x-playground


## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [3]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 10 max words, 2500 samples - at ../dataset/word-10-count.jsonl
Generated JSONL file with - 15 max words, 2500 samples - at ../dataset/word-15-count.jsonl
Generated JSONL file with - 20 max words, 2500 samples - at ../dataset/word-20-count.jsonl
Generated JSONL file with - 25 max words, 2500 samples - at ../dataset/word-25-count.jsonl
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/word-5-count.jsonl
Generated JSONL file with - 40 max words, 2500 samples - at ../dataset/word-40-count.jsonl
Generated JSONL file with - 50 max words, 2500 samples - at ../dataset/word-50-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-80-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-60-count.jsonl
Generated JSONL file with - 100 max words, 2500 sample

In [4]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-1.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/BaseV5-C-mem-finetune-1/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 329.74it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4a4930fde438f2a8/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 152.63it/s]


In [5]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-1.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-1.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-1 (bs=256, train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=512'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-1.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-1 (bs=256, train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--t

In [6]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/BaseV5-C-mem-finetune-1/last.ckpt" \
        "../model/BaseV5-C-Tune1.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/BaseV5-C-Tune1.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/BaseV5-C-mem-finetune-1/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/BaseV5-C-Tune1.pth
-rw-r--r-- 1 root root 5.7G Aug  7 01:13 ../model/BaseV5-C-Tune1.pth


In [7]:
# # Lets do a memory eval
# #
# # Note that the expected performance "is not that great", as the model seems to be only loosely
# # learning the memorization task, and the instruction propmt. And is seem to be acting more
# # like an RNG based on the instruct. (Instead of the actual memorization task)
# !python3 ../memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/BaseV5-C-Tune1.pth"

## Tune 2 : Low ctx size (512), memory training

- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.

In [8]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We switch over to fully masked instruct+input, to properly learn the memorization task
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
for i in {5..95..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 5000 & 
done
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-100-count.jsonl 100 5000 &
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-200-count.jsonl 200 5000 &

#
# We mixin the shuffled word list, so that we ensure all words / tokens are learned
# however this might intrduce an exclusion bias (if seen this word, never repeat it), 
# so we limit the mixture of this data samples
#
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-10-count.jsonl 10 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-15-count.jsonl 15 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-25-count.jsonl 25 30 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-50-count.jsonl 50 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-75-count.jsonl 75 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-100-count.jsonl 100 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-200-count.jsonl 200 50 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated a single JSONL file with 3570 samples (20 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 15 max words, 5000 samples - at ../dataset/gen-word-15-count.jsonl
Generated a single JSONL file with 3160 samples (30 token repeat) - 25 max words - at ../dataset/shuffle-word-25-count.jsonl
Generated a single JSONL file with 676 samples (50 token repeat) - 200 max words - at ../dataset/shuffle-word-200-count.jsonl
Generated JSONL file with - 20 max words, 5000 samples - at ../dataset/gen-word-20-count.jsonl
Generated a single JSONL file with 1323 samples (50 token repeat) - 100 max words - at ../dataset/shuffle-word-100-count.jsonl
Generated a single JSONL file with 1755 samples (50 token repeat) - 75 max words - at

In [9]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-2.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/BaseV5-C-mem-finetune-2/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 119.28it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ff571eccde34fd58/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 124.45it/s]


In [10]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-2.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-2 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-2.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-2 (bs=256, train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=512'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-2.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-2 (bs=256, train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--t

In [11]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/BaseV5-C-mem-finetune-2/last.ckpt" \
        "../model/BaseV5-C-Tune2.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/BaseV5-C-Tune2.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/BaseV5-C-mem-finetune-2/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/BaseV5-C-Tune2.pth
-rw-r--r-- 1 root root 5.7G Aug  7 02:24 ../model/BaseV5-C-Tune2.pth


In [12]:
# # Lets do a memory eval 
# #
# # While not at its full potential, its memory ability should start emerging
# #
# !python3 ../memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/BaseV5-C-Tune2.pth"

## Tune 3 : Ramping up the ctx size (1024), memory training

- Tune 3: Mid ctx size (1024), same as tune 2, but extended in context size

This intentionally a much larger dataset, and lower learning rate to help ensure we push the model to its absolute limits.

In [13]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 1000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

#
# Ramping up the 50+ - 400 words dataset
# 
for i in {50..450..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 1000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 10 max words, 1000 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 15 max words, 1000 samples - at ../dataset/gen-word-15-count.jsonl
Generated a single JSONL file with 656 samples (10 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated JSONL file with - 30 max words, 1000 samples - at ../dataset/gen-word-30-count.jsonl
Generated a single JSONL file with 1784 samples (10 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated a single JSONL file with 757 samples (10 token repeat) - 35 max words - at ../dataset/shuffle-word-35-count.jsonl
Generated JSONL file with - 5 max words, 1000 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 20 max words, 1000 samples - at ../dataset/gen-word-20-count.jsonl
Generated a single JSONL 

In [14]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-3.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/BaseV5-C-mem-finetune-3/"

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00, 25.36it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-367b06a2cc9f5220/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 73.35it/s]


In [15]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-3.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-3 (bs=256, train-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=1024

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-3.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-3 (bs=256, train-ctx=1024, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=1024'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-3.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-3 (bs=256, train-ctx=1024, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '

In [16]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/BaseV5-C-mem-finetune-3/last.ckpt" \
        "../model/BaseV5-C-Tune3.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/BaseV5-C-Tune3.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/BaseV5-C-mem-finetune-3/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/BaseV5-C-Tune3.pth
-rw-r--r-- 1 root root 5.7G Aug  7 05:01 ../model/BaseV5-C-Tune3.pth


In [17]:
# # Lets do a memory eval 
# #
# # We should start approaching the full potential of the model, unless its able to exceed 250 tokens of memory
# #
# !python3 ../memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/BaseV5-C-Tune3.pth"

## Tune 4 : Ramping up the ctx size (4096), memory training

- Tune 4: Mid ctx size (4096), same as tune 4, but extended in context size

This intentionally a much larger dataset, and lower learning rate to help ensure we push the model to its absolute limits.

In [18]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 2000 words dataset
# 
for i in {50..2000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 10 max words, 100 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 40 max words, 100 samples - at ../dataset/gen-word-40-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated a single JSONL file with 105 samples (1 token repeat) - 25 max words - at ../dataset/shuffle-word-25-count.jsonl
Generated a single JSONL file with 132 samples (1 token repeat) - 20 max words - at ../dataset/shuffle-word-20-count.jsonl
Generated a single JSONL file with 85 samples (1 token repeat) - 30 max words - at ../dataset/shuffle-word-30-count.jsonl
Generated a single JSONL file with 180 samples (1 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated a single JSONL file with 75 samples (1 token repeat) - 35 max words - at ../dataset/s

In [19]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-4.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/BaseV5-C-mem-finetune-4/"

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00,  6.01it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6b82aa37bdd9f9b2/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 39.30it/s]


In [23]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/BaseV5-C-mem-finetune-4.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-4 (bs=256, train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=4096

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-4.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-4 (bs=256, train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=4096'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/BaseV5-C/BaseV5-C-mem-finetune-4.yaml', '--trainer.logger.init_args.name=BaseV5-R2-C - Mem-Finetune-4 (bs=256, train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '

In [24]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/BaseV5-C-mem-finetune-4/last.ckpt" \
        "../model/BaseV5-C-Tune4.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/BaseV5-C-Tune4.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/BaseV5-C-mem-finetune-4/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/BaseV5-C-Tune4.pth
-rw-r--r-- 1 root root 5.7G Aug  8 07:39 ../model/BaseV5-C-Tune4.pth


In [22]:
# # Lets do a memory eval 
# !python3 ../memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/BaseV5-C-Tune4.pth"